# <p style="text-align: center;">MIS 382N: Advanced Predictive Modeling</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 50 </p>
## <p style="text-align: center;">Due: Mon, November 28</p>



Your homework should be written in a **Jupyter notebook**. Please submit **only one** ipynb file from each group, and include the names of all the group members. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). Board game data set from DataQuest will be used (you can download data from Canvas: 'games.csv').

1. (1) Load the data, (2) remove duplicate rows, (3) remove features of type string (object in pandas), and (4) replace missing values by mean of each column. Then, partition data into features (X) and the target label (y) for regression task. We want to predict the *average_rating*. Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) to predict average_rating. Find the best parameters (including *n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (4pts)

3. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Again, find the best parameters (including *n_estimators,* and* learning_rate*), and report corresponding RMSE for each algorithm. (8pts)

4. Which model did you expect to be more accurate in predicting the targets? Why? Did your observation match this expectation? (2pts)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)

### Part 1

In [2]:
#read data
game = pd.read_csv('games.csv')

#remove duplicate
game = game.drop_duplicates()

#drop string types
game = game.drop('name', 1)
game = game.drop('type', 1)
game = game.drop('id', 1)

#replace missing value
game = game.fillna(game.mean())

#spliting
y = game.average_rating
X = game.drop('average_rating', 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
from sklearn.metrics import fbeta_score, make_scorer

### Part 2

In [9]:
from sklearn.metrics import fbeta_score, make_scorer

rf = RandomForestRegressor()
parameters = {"n_estimators":[5,10,20,30],
              "max_depth": [3, None],
              "max_features": ['auto','sqrt','log2'],
              "min_samples_split": [3, 10],
              "min_samples_leaf": [3, 10]}

scorer = make_scorer(mean_squared_error, greater_is_better=False)
clf_rf = GridSearchCV(rf, parameters, cv = 5, scoring=scorer)
clf_rf.fit(X_train,y_train)

y_best_rf = clf_rf.predict(X_test)
RMSE_rf = np.sqrt(mean_squared_error(y_test, y_best_rf))

print clf_rf.best_params_
print 'RMSE: ', RMSE_rf

{'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 30, 'max_depth': None, 'min_samples_leaf': 3}
RMSE:  1.00767396657


### Part 3

In [4]:
#GBR
gbr = GradientBoostingRegressor(random_state=0, loss='ls')
param_grid = {'n_estimators': [10,50,100], 
              'learning_rate': [0.01, 0.1,1],
              'max_depth':[3,7,10],
             'min_samples_split':[5,10],
             'min_samples_leaf':[5,10]}
  

scorer = make_scorer(mean_squared_error, greater_is_better=False)
clf_gbr = GridSearchCV(gbr, param_grid, cv = 5, scoring=scorer)
clf_gbr.fit(X_train,y_train)

y_best_gbr = clf_gbr.predict(X_test)
RMSE_gbr = np.sqrt(mean_squared_error(y_test, y_best_gbr))

print clf_gbr.best_params_
print 'RMSE: ', RMSE_gbr

{'min_samples_split': 5, 'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 5}
RMSE:  0.999426023606


In [5]:
#AdaBoost
ada = AdaBoostRegressor()
param_grid = {'n_estimators': [10, 50, 100, 1000], 
              'learning_rate': [0.001,0.01, 0.1,1,10], 
             'loss':['linear', 'square', 'exponential']}
  

scorer = make_scorer(mean_squared_error, greater_is_better=False)
clf_ada = GridSearchCV(ada, param_grid, cv = 5, scoring=scorer)
clf_ada.fit(X_train,y_train)

y_best_ada = clf_ada.predict(X_test)
RMSE_ada = np.sqrt(mean_squared_error(y_test, y_best_ada))

print clf_ada.best_params_
print 'RMSE: ', RMSE_ada

{'n_estimators': 1000, 'loss': 'exponential', 'learning_rate': 0.01}
RMSE:  1.14109634194


### Part 4
In this case, boosting tree should perform better than Random Forest because we can tune the learning rate and achieve better accuracy. We also suspected that Adaboost wouldn't perform as well as GBR because Adaboost puts high weight on misclassifications, which makes it even more sensitive to outliers. The final results confirmed my hypothesis.

# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). [Spam Classification Data](https://archive.ics.uci.edu/ml/datasets/Spambase) of UCI will be used (you can download data from Canvas: 'spam_uci.csv'). Don't worry about column names. The last column represents target label, 1 if spam and zero otherwise.

1. Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify whether an email is spam. Find the best parameters (including *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

3. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

4. Point out one advantage and one disadvantage of Random Forest compared to GBDT (2pts)

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)

### Part 1

In [7]:
#read data
spam = pd.read_csv('spam_uci.csv')
print spam.head()

#spliting
y = spam['57']
X = spam.drop('57', 1)
X = X.drop('Unnamed: 0', 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

   Unnamed: 0     0     1     2    3     4     5     6     7     8 ...    48  \
0           0  0.00  0.64  0.64  0.0  0.32  0.00  0.00  0.00  0.00 ...  0.00   
1           1  0.21  0.28  0.50  0.0  0.14  0.28  0.21  0.07  0.00 ...  0.00   
2           2  0.06  0.00  0.71  0.0  1.23  0.19  0.19  0.12  0.64 ...  0.01   
3           3  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31 ...  0.00   
4           4  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31 ...  0.00   

      49   50     51     52     53     54   55    56  57  
0  0.000  0.0  0.778  0.000  0.000  3.756   61   278   1  
1  0.132  0.0  0.372  0.180  0.048  5.114  101  1028   1  
2  0.143  0.0  0.276  0.184  0.010  9.821  485  2259   1  
3  0.137  0.0  0.137  0.000  0.000  3.537   40   191   1  
4  0.135  0.0  0.135  0.000  0.000  3.537   40   191   1  

[5 rows x 59 columns]


### Part 2

In [8]:
rf = RandomForestClassifier(n_estimators=10)
parameters = {'n_estimators':[5,10,20,25,30],
              "max_depth": [3, None],
              "max_features": ['auto','sqrt','log2',None],
              "min_samples_split": [3, 10],
              "min_samples_leaf": [1, 3, 10],
              "criterion": ["gini", "entropy"]}

scorer = make_scorer(accuracy_score, greater_is_better=True)
clf_rf = GridSearchCV(rf, parameters, cv = 5, scoring=scorer)
clf_rf.fit(X_train,y_train)

y_best_rf = clf_rf.predict(X_test)
rf_prob = np.array([item[1] for item in clf_rf.predict_proba(X_test)])

print clf_rf.best_params_
print 'roc score:', roc_auc_score(y_test, rf_prob)
print 'accuracy score: ', accuracy_score(y_test, y_best_rf)

{'min_samples_leaf': 1, 'n_estimators': 30, 'min_samples_split': 3, 'criterion': 'entropy', 'max_features': 'sqrt', 'max_depth': None}
roc score: 0.988833673895
accuracy score:  0.953917050691


### Part 3

In [9]:
#GBDT
gbdt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0)
param_grid = {'n_estimators': [1, 10, 100, 1000], 'learning_rate': [0.001, 0.01, 0.1, 1, 10], 
              'max_depth': [3, 5, 10,15]}
  

scorer = make_scorer(accuracy_score, greater_is_better=True)
clf_gbdt = GridSearchCV(gbdt, param_grid, cv = 5, scoring=scorer)
clf_gbdt.fit(X_train,y_train)

y_best_gbdt = clf_gbdt.predict(X_test)

print clf_gbdt.best_params_
print 'roc score:', roc_auc_score(y_test, y_best_gbdt)
print 'accuracy score: ', accuracy_score(y_test, y_best_gbdt)

{'n_estimators': 1000, 'learning_rate': 0.1, 'max_depth': 3}
roc score: 0.95835874174
accuracy score:  0.961158657011


In [10]:
#Adaboost
ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.1)
param_grid = {'n_estimators': [1, 10, 50, 500], 'learning_rate': [0.01, 0.1, 1, 10, 100]}
  

scorer = make_scorer(accuracy_score, greater_is_better=True)
clf_ada = GridSearchCV(ada, param_grid, cv = 5, scoring=scorer)
clf_ada.fit(X_train,y_train)

y_best_ada = clf_gbdt.predict(X_test)
ada_prob = np.array([item[1] for item in clf_ada.predict_proba(X_test)])

print clf_ada.best_params_
print 'roc score:', roc_auc_score(y_test, ada_prob)
print 'accuracy score: ', accuracy_score(y_test, y_best_ada)

{'n_estimators': 500, 'learning_rate': 0.1}
roc score: 0.986291050178
accuracy score:  0.961158657011


### Part 4
Advantage: Random Forest is less likely to overfit to the data than GBDT because it's less susceptible to outliers.

Disadvantage: GBDT can have better accuracy with fewer trees as it tries to add new trees that compliment the already built ones

# Question 3 - Matrix Factorization for Rating Prediction (20pts)

The movielens dataset contains 1 million movie ratings from several thousand users. We will be using *k*-rank matrix factorization to estimate this dataset as the product $X=UV^T$, where *U* and *V* have only $k$ columns.

1) You can download the movielens 1M dataset from https://datahub.io/dataset/movielens, but for this problem use the data available on Canvas. It has been split into training and test sets, and converted to matrix format where the rows correspond to users and the columns to movies. Note that most of the entries are NaNs, indicating that these ratings are missing. An extra file, lens1m_361M_titles.csv, has been added so you can check out specific movies if you're curious.

2) Scikit-learn is a little behind for recommender systems, and doesn't have any method to factorize matrices with missing data. Which means you get to code it! Slide 22 of the 'apa large scale learning' lecture notes has the equations for stochastic gradient descent on *U* and *V*. You will have to:
* Set up initial guesses for the *U* and *V* matrices. I suggest small random values.
* Find a suitable learning rate for the descent. A learning rate that is too large will probably blow up, like in HW3 problem 1.
* Come up with a stopping policy
* Code the descent algorithm (5 pts)

3) Using your SGD algorithm, apply 2-rank matrix factorization on the filled training matrix. Calculate the RMSE of this model on the training data and on the test data (separately). The optimal score on the training data is around .86 RMSE; your version of gradient descent must go at least below .91 RMSE. (5 pts)

4) You should notice some overfitting. Because matrix factorization learns separate scores for each movie, a movie with very few reviews may be easily overfit. You may want to only predict ratings when you have enough information to reach a good conclusion. Recalculate the RMSE on the test data, specifically for movies with at least 50 reviews (don't retrain the models). Also report the percent of movies that are still included (after cutting those with < 50 reviews), and the percent of test ratings that are still included. (5 pts)

5) Repeat steps 3 and 4 with 5-rank factorization. Display training and test RMSE. (5 pts)

Hints:  
The numpy function *nanmean* is helpful for RMSE calculation.  
The descent algorithm will probably run for at least several minutes.

In [2]:
import numpy as np
import pandas as pd
titles = pd.read_csv('lens1m_361M_titles.csv')
test_X = np.load('lens1m_361M_test.npy')
train_X = np.load('lens1m_361M_train.npy')

In [14]:
train_X.shape

(6040L, 3952L)

### Part 2

In [158]:
class sgdMatrixFact():
    """Train a matrix factorization model to predict empty 
        entries in a matrix using stochastic gradient descent. 
        Params:
        K (int): number of latent features
        n_epochs (int): max number of epochs the model can run while training
        learning_rate (float): learning rate for SGD"""
    def __init__(self, K, n_epochs, learning_rate):
        self.K = K
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
    
    def fit(self,X_train):
        """The Fit function takes in a NxM matrix and returns 2 matrices, U and V, whose dot product
            is the approximation of the original matrix. It also returns a dictionary of the training
            RMSEs with each epoch"""
        self.X_train = X_train
        n, m = X_train.shape
        U = np.random.rand(n,self.K)/1000
        V = np.random.rand(m,self.K)/1000
        V = V.T #transpose V matrix
        
        train_rmse = {}
        for epoch in range(self.n_epochs):
            print "Working on epoch",epoch
            for i in xrange(len(X_train)):
                for j in xrange(len(X_train[i])):
                    if X_train[i][j]>0:
                        L = X_train[i,j] - np.dot(U[i,:],V[:,j])  # Calculate error for gradient
                        for k in range(self.K):
                            U[i][k] = U[i][k] + self.learning_rate*(2*L*V[k][j]) # Update latent U feature matrix
                            V[k][j] = V[k][j] + self.learning_rate*(2*L*U[i][k]) # Update latent V feature matrix
        
            X_hat = np.dot(U,V)
            rmse = np.sqrt(np.nanmean((X_train - X_hat)**2))
            train_rmse[epoch]=rmse
            try:
                if train_rmse[epoch-1]-rmse <= 0.001:
                    break #Stopping rule: break loop if rmse between epochs isn't improving
            except:
                pass
        
        self.train_rmse = train_rmse
        self.U = U
        self.V = V
        V = V.T #transpose V matrix back to normal
        return U,V,train_rmse
    
    def predict(self,X_test):
        """The Predict function uses the U and V matrices derived from the training matrix 
            and uses it to approximate empty entries in a testing matrix of the same shape. 
            Returns the approximated matrix and the test RMSE."""
        X_hat = np.dot(U,V.T)
        test_rmse = np.sqrt(np.nanmean((X_test - X_hat)**2))
        
        self.X_hat = X_hat
        self.test_rmse = test_rmse
        return X_hat,test_rmse      
            

In [161]:
#cross-validating to find an appropriate learning rate
learning_rate = np.linspace(0.01,0.001,15)
cross_val = {}
for l in learning_rate:
    mod = sgdMatrixFact(K=2,n_epochs = 5,learning_rate = l)
    U,V,rmse =  mod.fit(train_X)
    cross_val[l]=rmse

Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 0
Working on epoch 1
Working on epoch 0
Working on epoch 1
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 0
Working on e

In [162]:
cross_val

{0.001: {0: 3.7502882928430683,
  1: 3.4911177639934992,
  2: 1.8919557122839699,
  3: 1.3159876348936448,
  4: 1.1250026815000347},
 0.0016428571428571421: {0: 3.5218942176038257,
  1: 1.586507688548833,
  2: 1.1161458698564171,
  3: 1.0054637222580296,
  4: 0.96340231743312243},
 0.002285714285714285: {0: 2.8222951748141143,
  1: 1.1823327751550983,
  2: 0.99351126828159275,
  3: 0.95144317014240942,
  4: 0.93469069699268126},
 0.002928571428571428: {0: 2.4833238407006379,
  1: 1.0699252584477237,
  2: 0.95724239282006096,
  3: 0.93580132516690384,
  4: 0.92695756243234173},
 0.0035714285714285709: {0: 2.4848156798313736,
  1: 1.0238114509566973,
  2: 0.94360073104318409,
  3: 0.93064651498381845,
  4: 0.92512742380094037},
 0.0042142857142857138: {0: 2.5850792703714514,
  1: 1.0012069239775421,
  2: 0.9383192214743542,
  3: 0.92956146439740017,
  4: 0.9257629506615076},
 0.0048571428571428567: {0: 2.7524054523501684, 1: inf},
 0.0054999999999999997: {0: 2.9236414182641672, 1: inf},


### Part 3

In [163]:
#Since learning rate of 0.004 had the best RMSE, we use that, but increase the number of epochs to improve RMSE further.
#large number of epochs since stopping rule will take care of it if there are too many
mod = sgdMatrixFact(K=2,n_epochs = 20,learning_rate = 0.0042142857142857138) 
U,V,rmse =  mod.fit(train_X)
x_hat,test_rmse = mod.predict(test_X)

Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 5
Working on epoch 6
Working on epoch 7
Working on epoch 8
Working on epoch 9
Working on epoch 10
Working on epoch 11
Working on epoch 12
Working on epoch 13
Working on epoch 14
Working on epoch 15
Working on epoch 16
Working on epoch 17


In [164]:
print rmse
print test_rmse

{0: 2.5853193316908043, 1: 1.0011853781428373, 2: 0.93829741038596226, 3: 0.92952104296837912, 4: 0.925667316158444, 5: 0.92346245989240028, 6: 0.92167854163449592, 7: 0.91930932248581798, 8: 0.91564915684542592, 9: 0.91179442733142468, 10: 0.90900714891806744, 11: 0.90693997032273599, 12: 0.90524428415102121, 13: 0.90380068727552376, 14: 0.90254073581113137, 15: 0.90141054372764129, 16: 0.90036559843608943, 17: 0.89937042947662149}
0.924827123784


### Part 4

In [165]:
cols_to_use=[]
for col in range(test_X.shape[1]):
    if len(test_X[:,col][np.isfinite(test_X[:,col])])>=50:
        cols_to_use.append(col)

test_X_50 = test_X[:,cols_to_use]

In [166]:
x_hat_50 = x_hat[:,cols_to_use]
print "Test RMSE for Movies with over 50 reviews:", np.sqrt(np.nanmean((test_X_50 - x_hat_50)**2))

Test RMSE for Movies with over 50 reviews: 0.915069739888


In [167]:
print "Percent of movies still included: {:.2f}%".format((float(test_X_50.shape[1])/test_X.shape[1])*100)
print "Percent of ratings still included: {:.2f}%".format((float(len(test_X_50[np.isfinite(test_X_50)]))/len(test_X[np.isfinite(test_X)]))*100)

Percent of movies still included: 30.79%
Percent of ratings still included: 81.19%


### Part 5

In [168]:
mod1 = sgdMatrixFact(K=5,n_epochs = 20,learning_rate = 0.0042142857142857138) 
U1,V1,rmse1 =  mod.fit(train_X)
x_hat1,test_rmse1 = mod.predict(test_X)

Working on epoch 0
Working on epoch 1
Working on epoch 2
Working on epoch 3
Working on epoch 4
Working on epoch 5
Working on epoch 6
Working on epoch 7
Working on epoch 8
Working on epoch 9
Working on epoch 10
Working on epoch 11
Working on epoch 12
Working on epoch 13
Working on epoch 14
Working on epoch 15
Working on epoch 16
Working on epoch 17
Working on epoch 18
Working on epoch 19


In [169]:
print rmse1
print test_rmse1

{0: 2.5854680896037365, 1: 1.001295715580536, 2: 0.93829029998453817, 3: 0.92947816142040263, 4: 0.92553947385733237, 5: 0.92308247191871229, 6: 0.92059986910925562, 7: 0.91694246162152893, 8: 0.91254701585407461, 9: 0.90900427414003504, 10: 0.90624060918426452, 11: 0.90383807363295277, 12: 0.90167623333885671, 13: 0.89970543731292496, 14: 0.89788810789611018, 15: 0.89619784620985066, 16: 0.89462248907506814, 17: 0.89316249437276285, 18: 0.89182549450167592, 19: 0.89062017576247843}
0.924827123784


In [170]:
x_hat1_50 = x_hat1[:,cols_to_use]
print "Test RMSE for Movies with over 50 reviews:", np.sqrt(np.nanmean((test_X_50 - x_hat1_50)**2))

Test RMSE for Movies with over 50 reviews: 0.915069739888
